### Create the environment
To do so, we select an API endpoint and collect a token from our configuration. <br>
We also need an access token, in this example we use [oidc-agent](https://github.com/indigo-dc/oidc-agent#:~:text=oidc%2Dagent%20is%20a%20set,session%20or%20a%20login%20session.) to get one. <br>

In [1]:
eosc_perf_api="https://perf.test.fedcloud.eu/api/v1"
access_token=$(oidc-token egi-prod)

### (Conditional) Register, if not done already
To use our service as user, first we need to accept the terms of usage and register. <br>
Make sure to read the [terms and conditions](https://performance.services.fedcloud.eu/privacy_policy).

In [3]:
curl -X 'POST' \
  "$eosc_perf_api/users:register" \
  -H "Authorization: Bearer $access_token"

### Push a benchmark docker image at docker hub
All benchmark must rely on a public and accessible container image at docker hub.
> You can find a tutorial on how to push a docker container image to Docker Hub [here](https://docs.docker.com/docker-hub/repos/).

After upload your docker image, you will need the docker_image and docker_tag identifications for the later POST method.

In [2]:
image="deephdc/deep-oc-benchmarks_cnn"
tag="1.0.x-gpu-testing"

### Design a JSON Schema to accept or discard results from users
Results must be linked to a benchmark when submitted. 
You can control the required fields and their data types to ensure users do not upload invalid results.
This functionality will simplify users to compare attributes between results as such fields will always be present and will share the same type.
> If you do not want to use JSON Schemas, you can use `{}` for an always valid result.

> You can learn more about JSON Schemas at [json-schema.org](https://json-schema.org/).

In [3]:
schema='{
        "$id": "https://example.com/benchmark.schema.json",
        "$schema": "https://json-schema.org/draft/2019-09/schema",
        "type": "object",
        "properties": {
          "start_datetime": {
            "description": "The benchmark start datetime.",
            "type": "string",
            "format": "date-time"
          },
          "end_datetime": {
            "description": "The benchmark end datetime.",
            "type": "string",
            "format": "date-time"
          },
          "machine": {
            "description": "Execution machine details.",
            "type": "object",
            "properties": {
              "cpus": {
                "description": "Number of CPU.",
                "type": "integer"
              },
              "ram": {
                "description": "Available RAM in MB.",
                "type": "integer"
              }
            },
            "required": [
              "cpus",
              "ram"
            ]
          }
        },
        "required": [
          "start_datetime",
          "end_datetime",
          "machine"
        ]}'

### Upload your benchmark
To upload the benchmark, you only need to use an authenticated POST request to /benchmarks and attach the following content to the body:
- docker_image: Name of the image in docker hub.
- docker_tag: Tag of the docker image you want this benchmark to reference.
- schema: Defined JSON Schema to accept community results.
- description(Optional): Short description about the benchmark for the community users.

In [ ]:
curl -X 'POST' "$eosc_perf_api/benchmarks" \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $access_token" \
  -H 'Content-Type: application/json' \
  -d "{\"docker_image\": \"$image\", \"docker_tag\": \"$tag\", \"json_schema\": $schema, \
       \"description\": \"A free description for the community\"}" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
